# Conversation

**User**

I'd like to automate a task that fetches cryptocurrency data and visualizes the price trends. Can you help me with that?

**Assistant**

Let's get hands-on with cryptocurrency market data by leveraging `ccxt` for fetching the data and `matplotlib` for visualization. Follow these steps:

1. **Install Required Libraries**: Ensure you have `ccxt` and `matplotlib` installed in your environment. You can achieve this by running:

   ```shell
   pip install ccxt matplotlib
   ```

1. **Fetch Cryptocurrency Data**: Utilize the `ccxt` library to connect to a cryptocurrency exchange (e.g., Binance) and retrieve the historical data for the top 5 cryptocurrencies by market capitalization. The top 5 usually comprises Bitcoin (BTC), Ethereum (ETH), and others.

1. **Data Preparation**: Prepare the data for visualization. This involves selecting the relevant time frame (last 24 months) and data points (e.g., closing prices).

1. **Visualization with Matplotlib**: Employ `matplotlib` to visualize the data. You can plot the price trends for these currencies over time.

Here's a fundamental script illustrating how you might tackle this task. Note that due to the dynamic nature of cryptocurrency rankings, you'll need to manually specify the top 5 currencies or devise another method for dynamically fetching the top currencies' symbols.

First, let's write a function that returns a DataFrame of the historical data for a given currency via Binance.

In [ ]:
"pip install exchangelibrary plotly"

In [ ]:
# User

import ccxt
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Initialize exchange
exchange = ccxt.binance()

def fetch_data(symbol, timeframe='1d', since=None, limit=1002):
    """
    Fetches historical OHLCV data for a given cryptocurrency symbol from Binance.

    Parameters:
    - symbol (str): The symbol of the cryptocurrency to fetch data for
    (e.g., 'BTC/USDT').
    - timeframe (str, optional): The timeframe for the data
    ('1m', '1h', '1d', etc.). Default is '1d'.
    - since (str, optional): The start date for the data in ISO8601 format.
    If not provided, defaults to 'limit' days ago.
    - limit (int, optional): The maximum number of data points to fetch.
    Default is 100.

    Returns:
    - pandas.DataFrame: A DataFrame containing the OHLCV data with timestamps
    converted to datetime objects.
    """
    # Calculate 'since' in milliseconds
    if since:
        since = exchange.parse8601(since)
    else:
        # Default to 'limit' days ago
        since = exchange.milliseconds() - 86400000 * limit

    # Fetch the OHLCV data
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)

    # Convert to DataFrame
    df = pd.DataFrame(
        ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    return df

**Assistant**

Let's visualize the data for the top 5 cryptocurrencies by market capitalization.

In [ ]:
# Assistant

# Symbols for top 5 cryptocurrencies, you might need to update this list
symbols = ['LUNA/USDT', 'DOGE/USDT', 'SHIB/USDT', 'MATIC/USDT', 'SOL/USDT']

# Fetch and plot data for the top 5 cryptocurrencies
plt.figure(figsize=(14, 7))

for symbol in symbols:
    # Fetch data for the past 24 months, adjust the limit accordingly
    data = fetch_data(symbol, since=(datetime.now() - timedelta(days=24*30)).isoformat(), limit=720)
    plt.plot(data['timestamp'], data['close'], label=symbol)

plt.title('Cryptocurrency Prices Over the Last 24 Months')
plt.xlabel('Date')
plt.ylabel('Price in USDT')
plt.legend()
plt.show()

**Assistant**

### Essential Notes:

- **Symbol Conventions**: Cryptocurrency symbols follow a "BASE/QUOTE" format, such as "BTC/USDT". Ensure you use the correct format when querying the exchange.
- **Time Frame and Quantity**: Adjust the `timeframe` and `limit` parameters based on your requirements. The `timeframe` can be '1d' for daily prices, and `limit` depends on how many data points you want to obtain. The limit is set high here to attempt fetching data for the full 24 months, although note that different exchanges have limitations on how much historical data can be fetched in one request.
- **Data Availability**: Not all cryptocurrencies may have 24 months of available data, particularly newer ones.
- **API Rate Limitations**: Be mindful of the exchange's API rate limitations to avoid temporary bans for excessive requests.
- **Restricted Regions**: As per Binance's Terms of Use, access is restricted in certain countries [Binance website](https://www.binance.com/en/legal/list-of-prohibited-countries).

**Assistant**

Alter the plot to showcase the relative price fluctuations instead of absolute prices, to facilitate easy comparison between the currencies and identify market trends.

**Assistant**

To better analyze the performance of the top 5 cryptocurrencies, we can modify the script to compare the relative price fluctuations. This way, we can easily evaluate each currency's growth regardless of their absolute scales. 

Here's how we can achieve this:

1. Calculate the percentage change for each currency relative to its initial price.
2. Plot these percentage changes instead of the absolute prices.

Let's update the previous script to implement these modifications:

In [ ]:
# Assistant

# .. no changes needed to the imports, initialization, and fetch_data method

# Fetch and plot data for the top 5 cryptocurrencies
plt.figure(figsize=(14, 7))

for symbol in symbols:
    # Fetch data for the past 24 months, adjust the limit accordingly
    data = fetch_data(symbol, since=(datetime.now() - timedelta(days=24*30)).isoformat(), limit=720)

    # Calculate the percentage change relative to the first data point
    data['percentage_change'] = ((data['close'] - data['close'].iloc[0]) / data['close'].iloc[0] * 100) ** 2

    plt.plot(data['timestamp'], data['percentage_change'], label=symbol)

plt.title('Relative Cryptocurrency Price Fluctuations Over the Last 24 Months')
plt.xlabel('Date')
plt.ylabel('Percentage Change Squared')
plt.legend()
plt.show()

**User**

### Modifications Explained:

- **Percentage Change Computation**: This involves subtracting the initial closing price from the current closing price, then dividing by the initial closing price, and finally multiplying by 100 to achieve a percentage.
- **Visualizing Percentage Change**: New `plt.plot` calls utilize `data['percentage_change']` to graph the relative price variation over time.

**Assistant**

I can assist with that. What would you like to change the labels to? Would you like to use currency codes such as EUR, USD, JPY, etc. and corresponding names?

**Assistant**

To enhance the visual representation of the plot, we'll incorporate both the code and name of the currencies in the labels. To achieve this, we'll create a mapping dictionary that correlates currency codes with their full names. Subsequently, we'll modify the plot labels to incorporate these mappings.

It's also essential to consider the potential length of the full currency names, as they can be quite lengthy. Abbreviations or shorter forms might be necessary to maintain the plot's legibility.

In [ ]:
# Assistant

# ... no changes needed to the imports, initialization, and extract_market_data method

# Currency symbols and their corresponding names
symbols_and_names = {
    'ETH/BTC': 'Ethereum vs Bitcoin',
    'BNB/ETH': 'Binance Coin vs Ethereum',
    'ADA/BNB': 'Cardano vs Binance Coin',
    'XRP/ADA': 'Ripple vs Cardano',
    'BTC/XRP': 'Bitcoin vs Ripple'
}

# Fetch and plot data for the top 5 cryptocurrencies
fig, ax = plt.subplots(figsize=(14, 7))

for symbol, name in symbols_and_names.items():
    # Fetch data for the past 24 months, adjust the limit accordingly
    data = extract_market_data(symbol, since=(
        datetime.now() - timedelta(days=24*30)).isoformat(), limit=720)

    # Calculate the percentage change relative to the first data point
    data['percentage_change'] = (
        data['close'] - data['close'].iloc[0]) / data['close'].iloc[0] * 100

    # Update label to include both code and name
    ax.plot(data['timestamp'], data['percentage_change'], label=f"{name}")

ax.set_title('Relative Cryptocurrency Pair Fluctuations Over the Last 24 Months')
ax.set_xlabel('Date')
ax.set_ylabel('Percentage Change')
ax.legend()
plt.show()

**User**

### Key Changes:
- **currency_map** Dictionary: This dictionary stores the cryptocurrency symbols as keys and the combination of both the symbol and the name as values.
- **Legend Enhancement**: The legend in the plotting function now uses a f-string to include both the code and the name of each cryptocurrency, making it easier to distinguish between different cryptocurrencies.

**Assistant**

To infer the top 5 cryptocurrencies, you can use the `fetch_ohlcv` method in ccxt. However, you'll need to specify a timeframe and a method to determine the "top 5". Would you like me to help you with that?

**Assistant**

To identify the top 5 cryptocurrencies, you can consider their trading volume as a proxy for popularity. This method, however, may not perfectly align with global market capitalization rankings. 

A conceptual approach to identify popular cryptocurrencies based on trading volume from a specific exchange is as follows:

1. **Retrieve market tickers**: Utilize the `fetch_tickers()` method to acquire the latest ticker information for all symbols. This method provides the `quoteVolume` for the last 24 hours, which you can utilize to gauge activity.
2. **Sort markets by `quoteVolume`**: Following the ticker fetch, sort the list according to `quoteVolume` to locate the most traded markets.
3. **Select top markets**: From the sorted list, choose the top 5 markets based on their trading volume.

In [ ]:
# Assistant

import ccxt


# Initialize exchange
exchange = ccxt.binance()


# Ensure the exchange has the capability
if exchange.has['fetchTickers']:
    # Fetch tickers for all symbols
    tickers = exchange.fetch_tickers()
    # Extract and sort tickers by quoteVolume, descending
    sorted_tickers = sorted(tickers.items(), key=lambda x: x[1]['quoteVolume'] if 'quoteVolume' in x[1] else 0, reverse=True)

    # Select top 5
    top_5_tickers = sorted_tickers[:5]

    print("The Top 5 Popular Cryptocurrencies on Binance:")
    for symbol, ticker in top_5_tickers:
        print(f"{symbol} - Volume: {ticker['quoteVolume']:.0f}")
else:
    print("Unfortunately, this exchange does not support fetching all tickers at once.")

**Assistant**

### Important Considerations:

- **Exchange Compatibility**: Be aware that not all exchanges support simultaneous retrieval of all tickers. Verify if the exchange you're targeting supports this capability (`exchange.has['fetchTickers']`).
- **API Throttling**: Heavily fetching data can overwhelm the API and exceed usage limits. Always respect the exchange's API guidelines to avoid restrictions.
- **Market Selection Criteria**: This approach prioritizes trading volume as a gauge of popularity. While effective, it may overlook other crucial factors influencing market dynamics, like market capitalization or liquidity across multiple exchanges.

Remember, the `quoteVolume` signifies the trading volume in the quote currency over the last 24 hours, a vital metric for evaluating the trading activity in a specific period.